<a href="https://colab.research.google.com/github/HanqiLouis/GFET-Characterization/blob/main/Plot_all_Runs_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# @title Plot all Runs
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output


### ------- Plot Functions -------
def plot_site_runs(directory_paths, folders_names, site_number, runs_per_site=3, dual_sweep=False, save_output=None, display=False):
    """
    Plots the runs for a specific site for multiple folders and optionally saves the plot.

    Parameters:
    - directory_paths: list of str, paths to the folders containing Excel files.
    - folders_names: list of str, names of the folders for labeling.
    - site_number: int, the site to plot (1-indexed).
    - runs_per_site: int, number of runs per site (default: 3).
    - dual_sweep: bool, if the measurement mode was dual, set dual_sweep to True to only plot the forward part.
    - save_output: str or None, path to the folder where the plot should be saved (default: None).
    """
    # Helper function to get files for a site from a folder
    def get_site_files(directory_path):
        start_index = (site_number - 1) * runs_per_site
        end_index = start_index + runs_per_site
        excel_files = sorted(glob.glob(os.path.join(directory_path, '*.xlsx')))
        return excel_files[start_index:end_index]

    plt.figure(figsize=(8, 6))

    # Iterate through all provided folders
    for folder_index, directory_path in enumerate(directory_paths, start=1):
        site_files = get_site_files(directory_path)

        if not site_files:
            print(f"No files found for Site {site_number} in folder {folder_index}.")
            continue

        # Plot runs from the current folder
        for run_index, file in enumerate(site_files, start=1):
            df = pd.read_excel(file)
            if dual_sweep:
                total_rows = df.shape[0]
                df_subset = df.iloc[0:int(total_rows / 2)]
                Vg = df_subset['VG']
                Id = df_subset['ID']
            else:
                Vg = df['VG']
                Id = df['ID']
            plt.plot(
                1000 * Vg,
                1000000 * Id,
                marker='.',
                markersize=1,
                linestyle='-',
                label=f'{folders_names[folder_index-1]} Run {run_index}',
                linewidth=1
            )

    # Add labels, title, and legend
    plt.xlabel('VG [mV]')
    plt.ylabel('ID [uA]')
    plt.title(f'Site {site_number}')
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1))
    plt.tight_layout()
    plt.grid()

    # Save the plot if save_output is specified
    if save_output is not None:
        os.makedirs(save_output, exist_ok=True)  # Create the directory if it doesn't exist
        plot_path = os.path.join(save_output, f'site_{site_number}_plot.png')
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to {plot_path}")

    # Display the plot if display is True
    if display:
      plt.show()
    else:
      plt.close()


### ------- Global Variables -------
folder_path_widgets = []  # List of text fields for folder paths
folder_name_widgets = []  # List of text fields for folder names
output = widgets.Output()  # Output display for results

### ------- Folder Management Functions -------
def add_folder(_=None):
    """ Adds a new folder input set (path & name) dynamically """
    index = len(folder_path_widgets) + 1

    path_widget = widgets.Text(
        description=f'Path to Measurement Folder {index}:',
        placeholder=f'Enter path for Measurement Folder {index}',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='750px')
    )

    name_widget = widgets.Text(
        description=f'Measurement {index} Name:',
        placeholder=f'Enter name for Measurement {index}',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='350px')
    )

    folder_path_widgets.append(path_widget)
    folder_name_widgets.append(name_widget)
    update_ui()

def remove_folder(_):
    """ Removes the last added folder input set, ensuring at least one remains """
    if len(folder_path_widgets) > 1:
        folder_path_widgets.pop()
        folder_name_widgets.pop()
        update_ui()

### ------- Buttons for Adding & Removing Folders -------
add_button = widgets.Button(description="+", button_style="success")
remove_button = widgets.Button(description="-", button_style="danger")

add_button.on_click(add_folder)
remove_button.on_click(remove_folder)

### ------- Site & Run Fields -------
site_field = widgets.BoundedIntText(
    value=0, min=0, description='Number of sites:', style={'description_width': 'initial'}
)

run_field = widgets.BoundedIntText(
    value=0, min=0, description='Runs per site:', style={'description_width': 'initial'}
)

### ------- Dropdown for Site Selection -------
dropdown = widgets.Dropdown(options=[0], value=0, description='Site to display:', style={'description_width': 'initial'})

def update_dropdown_options(change):
    """ Updates dropdown options based on site_field value """
    dropdown.options = [i for i in range(1, change['new'] + 1)]

site_field.observe(update_dropdown_options, names='value')

### ------- Checkbox for Dual Sweep -------
dual_sweep_checkbox = widgets.Checkbox(value=False, description='Dual sweep', style={'description_width': 'initial'})

### ------- Save Functionality -------
save_checkbox = widgets.Checkbox(value=False, description='Save', style={'description_width': 'initial'})

save_directory_field = widgets.Text(
    description='Save Directory:',
    placeholder='Enter directory to save files',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='750px')
)

### ------- Plot Runs Function -------
def plot_runs(_):
    """ Processes input values and calls plot function """
    folder_path = []
    folder_name = []

    with output:
        clear_output()

        if site_field.value == 0:
            print("Please enter the number of sites")
            return
        if run_field.value == 0:
            print("Please enter the number of runs per site")
            return

        # Count the number of provided folders
        j = sum(1 for widget in folder_path_widgets if widget.value.strip())

        if j == 0:
            print("No folder provided")
            return

        # Collect paths & names
        for i in range(j):
            folder_path.append(folder_path_widgets[i].value.strip())
            folder_name.append(folder_name_widgets[i].value.strip() or f'Refiling{i+1}')

        # Call the external function plot_site_runs()
        if save_checkbox.value:
            if not save_directory_field.value.strip():
                print("Please enter the save directory")
                return
            else:
                plot_site_runs(
                    folder_path, folder_name, site_number=dropdown.value,
                    runs_per_site=run_field.value, dual_sweep=dual_sweep_checkbox.value,
                    save_output=save_directory_field.value.strip(), display=True
                )
        else:
            plot_site_runs(
                folder_path, folder_name, site_number=dropdown.value,
                runs_per_site=run_field.value, dual_sweep=dual_sweep_checkbox.value,
                save_output=None, display=True
            )

### ------- Save All Plots Function -------
def save_all_plots(_):
    """ Saves all plots to a specified directory """

    folder_path = []
    folder_name = []

    with output:
        clear_output()

        if not save_directory_field.value.strip():
            print("Please enter the save directory")
            return

        j = sum(1 for widget in folder_path_widgets if widget.value.strip())

        if j == 0:
            print("No folder provided")
            return

        if site_field.value == 0:
            print("Please enter the number of sites")
            return
        if run_field.value == 0:
            print("Please enter the number of runs per site")
            return
                # Collect paths & names
        for i in range(j):
            folder_path.append(folder_path_widgets[i].value.strip())
            folder_name.append(folder_name_widgets[i].value.strip() or f'Refiling{i+1}')

        for i in range(site_field.value):
            plot_site_runs(
                folder_path, folder_name, site_number=i+1,
                runs_per_site=run_field.value, dual_sweep=dual_sweep_checkbox.value,
                save_output=save_directory_field.value.strip(), display=False
            )

        print("All plots saved in:", save_directory_field.value.strip())

### ------- Buttons for Plotting & Saving -------
plot_button = widgets.Button(description='Plot', button_style='warning')
plot_button.on_click(plot_runs)

save_all_button = widgets.Button(description='Save all plots', button_style='primary')
save_all_button.on_click(save_all_plots)

### ------- UI Update Function (Defined at the End) -------
def update_ui():
    """ Clears and updates the UI with the latest state of widgets """
    clear_output(wait=True)

    # Display all dynamic folder fields
    for path, name in zip(folder_path_widgets, folder_name_widgets):
        display(name)
        display(path)

    # Display action buttons
    display(widgets.HBox([add_button, remove_button]))

    # Display the rest of the widgets
    display(site_field, run_field)
    display(dropdown)
    display(dual_sweep_checkbox, save_checkbox)
    display(save_directory_field)
    display(plot_button, output)
    display(save_all_button)

# Initialize the first folder field AFTER defining everything
add_folder(None)
update_ui()

Text(value='', description='Measurement 1 Name:', layout=Layout(width='350px'), placeholder='Enter name for Me…

Text(value='', description='Path to Measurement Folder 1:', layout=Layout(width='750px'), placeholder='Enter p…

BoundedIntText(value=0, description='Number of sites:', style=DescriptionStyle(description_width='initial'))

BoundedIntText(value=0, description='Runs per site:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Site to display:', options=(0,), style=DescriptionStyle(description_width='initial'), va…

...

Checkbox(value=False, description='Save', style=DescriptionStyle(description_width='initial'))

Text(value='', description='Save Directory:', layout=Layout(width='750px'), placeholder='Enter directory to sa…

Button(button_style='warning', description='Plot', style=ButtonStyle())

Output()

Button(button_style='primary', description='Save all plots', style=ButtonStyle())